In [395]:
import math
import random
import pandas as pd
import numpy as np

In [396]:
# Funcao Objetivo
def f_alpine02(x):
    resultado = 1
    for i, x_i in x.iteritems():
        resultado = resultado * math.sin(x_i) * math.sqrt(x_i)
    return resultado

In [397]:
# Ordenar
def ordena(df):
    df['aux'] = df.apply(f_alpine02, axis=1)
    df = df.sort_values('aux', ascending=False).drop('aux', axis = 1)
    return df.reset_index(drop=True)

In [398]:
def ranking_linear(df, max, min):
    Fx = []
    df_ordenado = ordena(df)
    df_ordenado = df_ordenado.reset_index(drop = True)
    for i, row in df_ordenado.iterrows():
        resultado = min + (max - min)*(( len(df_ordenado) - i - 1)/( len(df_ordenado) -1))
        Fx.append(resultado)
    return Fx

In [399]:
# Mutacao
def mutacao_uniforme(individuo, tx_mutacao):
    individuo_mutado = []
    # como a taxa de mutacao eh entre 0 e 1, temos:
    for x_i in individuo:
        rand = np.random.random()
        
        if rand < tx_mutacao:
            x_i = np.random.uniform(0.0, 10.0)
        individuo_mutado.append(x_i)
    return individuo_mutado

In [400]:
def crossover(P1, P2):
    C1 = P1
    C2 = P2
    alpha = np.random.random()
    #print('olha ele aqui ----', alpha)
    for i in range(len(P1)):
        t = alpha*P1[i] + (1 - alpha)*P2[i]
        s = alpha*P2[i] + (1 - alpha)*P1[i]

        if t>=0 and t<=10 and s<=10 and s>=0:
            C1[i] = t
            C2[i] = s
    return (C1, C2)

In [401]:
# Netodo de selecao - roleta
def selecao_roleta(Fx, df):
    posicao = 0
    soma_acumulada = np.cumsum(Fx)
    tamanho = len(Fx)
    limite = soma_acumulada[tamanho-1]
    rand = np.random.uniform(0.0, limite)

    for i, valor in enumerate(soma_acumulada):
        if rand <= valor:
            posicao = i
            break
    return df.loc[posicao]

In [403]:
# define tamanho da populacao
tamanho_pop: int = 100
tx_mutacao = 0.005

# Cria dataframe e preenche com valores randomicos
populacao = pd.DataFrame(np.random.uniform(0.0, 10.0, size=(tamanho_pop, 2)), columns=['X1', 'X2'])

#print(populacao)

# melhor
best = []
pior = []
melhor = []
media = []
# num geracoes
total_geracoes = 20
geracao = 0
while (geracao < total_geracoes) : 
    # ordena
    populacao_ordenada = ordena(populacao).reset_index(drop = True)
    print(populacao_ordenada)
    # Pegar melhor individuo da geracao atual
    best = populacao_ordenada.loc[0]
    melhor.append(best)
    pior.append(populacao_ordenada.loc[99])
    #media.append()

    Fx = ranking_linear(populacao_ordenada, 10, 0)
    populacao_ordenada = populacao_ordenada.drop('aux', axis=1)

    nova_populacao = pd.DataFrame(columns = ['X1', 'X2'])
    for i in range(0, int(tamanho_pop/2)):
        # chama selecao
        P1 = selecao_roleta(Fx, populacao_ordenada)
        P2 = selecao_roleta(Fx, populacao_ordenada)
     
        # transforma P1 e P2 em vetor
        P1 = P1.to_numpy()
        P2 = P2.to_numpy()

        # faz crossover
        C1, C2 = crossover(P1, P2)

        # Realizar mutacao de C1 e C2
        C1 = mutacao_uniforme(C1, tx_mutacao)
        C2 = mutacao_uniforme(C2, tx_mutacao)
        
        nova_populacao=nova_populacao.append(pd.DataFrame(data=[C1, C2], columns = ['X1', 'X2'] ))
 
    populacao = nova_populacao.reset_index(drop=True)
    geracao += 1

print(f_alpine02(best))
'''for i in melhor:
    print(f_alpine02(i))
print()
for i in pior:
    print(f_alpine02(i))'''

          X1        X2
0   7.589011  7.713778
1   8.164828  7.332240
2   8.251637  8.448433
3   7.321085  7.381177
4   4.713415  4.553463
..       ...       ...
95  5.228908  8.429438
96  5.086576  8.148241
97  8.236433  4.760475
98  4.928954  7.978851
99  8.046018  4.821237

[100 rows x 2 columns]
          X1        X2
0   7.653927  7.531593
1   8.393327  8.378134
2   7.423131  7.304230
3   7.328792  7.363897
4   8.754472  7.238971
..       ...       ...
95  7.324378  4.676863
96  7.620004  5.322959
97  8.010598  4.228659
98  4.484858  7.853086
99  4.686379  8.146862

[100 rows x 2 columns]
          X1        X2
0   7.400806  7.967590
1   8.583079  7.602282
2   4.856001  4.556159
3   4.586827  4.675519
4   5.166293  5.094056
..       ...       ...
95  5.568092  8.442155
96  4.350459  8.651402
97  7.336316  4.185930
98  7.222696  4.379853
99  7.191196  4.706890

[100 rows x 2 columns]
          X1        X2
0   4.797770  5.071462
1   4.730168  4.483166
2   4.435179  4.878643
3   5.09

'for i in melhor:\n    print(f_alpine02(i))\nprint()\nfor i in pior:\n    print(f_alpine02(i))'